In [245]:
import torch, pickle
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from collections import Counter


---

## Create EMG Data Loader

### Load Dataset Pickle

In [2]:
file = open('df_hudgin.pkl', 'rb')
df = pickle.load(file)
file.close()

In [3]:
df.head()

,subject,day,session,motion,repetition,window_emg,mav,wl,ssc,zc
0,sub03,D3,S1,OH,1,"[[0.4140625, 0.6015625, -0.203125, -0.2109375,...","[0.26875, 0.1434375, 0.17390625, 0.03375, 0.03...","[18.875, 10.3515625, 14.3125, 2.296875, 2.1562...","[29, 27, 40, 29, 26, 29, 34, 42]","[24, 20, 30, 27, 18, 25, 35, 30]"
1,sub03,D3,S1,OH,1,"[[0.2421875, -0.0546875, -0.0078125, 0.015625,...","[0.2403125, 0.09796875, 0.135, 0.0196875, 0.02...","[18.3125, 6.9453125, 12.1015625, 1.4375, 2.218...","[33, 31, 41, 29, 33, 28, 37, 34]","[28, 19, 35, 21, 26, 25, 36, 28]"
2,sub03,D3,S1,OH,1,"[[-0.5546875, -0.125, -0.0625, -0.0234375, -0....","[0.153125, 0.1021875, 0.130625, 0.01890625, 0....","[11.6640625, 7.6328125, 12.078125, 1.3125, 1.9...","[31, 31, 40, 29, 29, 25, 37, 25]","[29, 26, 37, 18, 25, 22, 29, 24]"
3,sub03,D3,S1,OH,1,"[[-0.1953125, -0.0625, -0.265625, -0.0234375, ...","[0.12859375, 0.10546875, 0.10609375, 0.0170312...","[9.609375, 8.34375, 8.875, 1.140625, 1.375, 5....","[30, 32, 37, 34, 22, 28, 34, 30]","[30, 27, 32, 17, 16, 24, 26, 28]"
4,sub03,D3,S1,OH,1,"[[0.0703125, -0.1484375, 0.171875, -0.015625, ...","[0.13140625, 0.1171875, 0.08609375, 0.01421875...","[9.59375, 9.6875, 6.8046875, 0.9296875, 1.1796...","[33, 32, 35, 33, 25, 32, 33, 35]","[26, 32, 28, 15, 16, 26, 30, 29]"


In [209]:
np.arange(0, 10).tolist()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [213]:
df['window_emg_list'] = df.window_emg.apply(lambda x: x.tolist())

In [216]:
del df['window_emg_list']

In [214]:
file = open('df_hudgin_min.pkl', 'wb')
pickle.dump(df[['subject', 'day', 'session', 'motion', 'repetition', 'window_emg_list']], file)
file.close()

### Preprocess Labels

In [4]:
df['label'] = df.motion.replace(
    df.motion.drop_duplicates().tolist(),
    [i for i in range(9)]
)

/var/folders/fg/wwsz02rn0qn31sz3qslnnjm80000gn/T/ipykernel_66414/4281264563.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['label'] = df.motion.replace(


In [5]:
df.label.value_counts()

label
0    65100
1    65100
2    65100
3    65100
4    65100
5    65100
6    65100
7    65100
8    65100
Name: count, dtype: int64

## Create Model Architecture

In [56]:
class Autoencoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.Sigmoid()
        )
        self.decoder = nn.Sequential(
            nn.Linear(hidden_size, input_size),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [57]:
class SSAE(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(SSAE, self).__init__()
        self.ae1 = Autoencoder(input_dim, hidden_dim1)
        self.ae2 = Autoencoder(hidden_dim1, hidden_dim2)
        self.classifier = nn.Linear(hidden_dim2, output_dim)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        encoded1, _ = self.ae1(x)
        encoded2, _ = self.ae2(encoded1)
        logits = self.classifier(encoded2)
        output = self.softmax(logits)
        
        return output

In [65]:
ssae.ae1.encoder[1].parameters()

<generator object Module.parameters at 0x177e58d60>

In [77]:
a = torch.rand(1, 8, 4)

In [79]:
e, d = ssae.ae1(a)

In [80]:
loss_fn = SSAELoss()

In [81]:
loss_fn(a, d, e, ssae.ae1)

tensor(1.5382, grad_fn=<AddBackward0>)

In [157]:
class SSAELoss(nn.Module):
    def __init__(self, l2_weight=0.0001, sparsity_weight=0.01, sparsity_target=0.05):
        super(SSAELoss, self).__init__()
        self.l2_weight = l2_weight
        self.sparsity_weight = sparsity_weight
        self.sparsity_target = sparsity_target
        self.mse_loss = nn.MSELoss(reduction='mean')
        self.kl_loss = nn.KLDivLoss(reduction='sum')

    def forward(self, x, decoded, encoded, model):
        # Mean Square Error loss
        mse_loss = self.mse_loss(decoded, x)

        # L2 regularization loss
        l2_loss = 0
        for param in model.parameters():
            l2_loss += torch.sum(param ** 2)
        l2_loss *= self.l2_weight * .5

        # KL divergence sparsity loss
        rho_hat = torch.mean(encoded, dim=0)
        rho = torch.full_like(rho_hat, self.sparsity_target)
        
        # Prepare inputs for KLDivLoss
        p = torch.stack([rho, 1 - rho])
        q = torch.stack([rho_hat, 1 - rho_hat])
        
        # Calculate KL divergence
        kl_loss = self.kl_loss(torch.log(q), p)

        # Total loss
        # print(mse_loss.tolist(), l2_loss.tolist(), kl_loss.tolist())
        total_loss = mse_loss + l2_loss + self.sparsity_weight * kl_loss

        return total_loss

In [225]:
1e-2

0.01

In [284]:
def train_autoencoder(autoencoder, data_loader, num_epochs, lr=0.01, patience=10):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    autoencoder = autoencoder.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(autoencoder.parameters(), lr=lr, momentum=0.9)

    # For saving best model
    best_train_loss = float('inf')
    best_model = None
    epochs_no_improve = 0
    epsilon = 1e-2
    
    for epoch in range(num_epochs):
        total_loss = 0
        for i, batch in enumerate(data_loader, 0):
            inputs = batch[0].to(device)
            
            optimizer.zero_grad()
            encoded, decoded = autoencoder(inputs)
            loss = criterion(inputs, decoded)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        # total_loss /= len(data_loader)
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.6f}')
        
        # Check if model is better
        if best_train_loss - total_loss > epsilon:
            best_train_loss = total_loss
            best_model = autoencoder.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
        
        # Check early stopping condition
        if epochs_no_improve == patience:
            print("Early stopping!")
            break

    # Load best model
    autoencoder.load_state_dict(best_model)

    return best_model

In [286]:
def train_ssae(ssae, train_loader, num_epochs, lr=0.01, batch_size=256):
    device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
    ssae = ssae.to(device)

    # Greedy layer-wise training
    ## for the first layer
    print('Training autoencoder 1')
    ae1_best = train_autoencoder(ssae.ae1, train_loader, num_epochs, lr)
    ssae.ae1.load_state_dict(ae1_best)

    ## prepare data for the next autoencoder
    encoded_train_loader = DataLoader(
        TensorDataset(
            torch.cat([ssae.ae1.encoder(batch[0].to(device)).detach() for batch in train_loader])
        ),
        batch_size=batch_size,
        shuffle=True
    )

    ## for the second layer
    print('Training autoencoder 2')
    ae2_best = train_autoencoder(ssae.ae2, encoded_train_loader, num_epochs, lr)
    ssae.ae2.load_state_dict(ae2_best)
        
    # Fine-tuning
    print("Fine-tuning the entire network")

    # For saving best model
    # best_train_loss = float('inf')
    # best_model = None
    # epochs_no_improve = 0
    # epsilon = 1e-2
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(ssae.parameters(), lr=lr, momentum=0.95)

    for epoch in range(num_epochs):
        ssae.train()
        
        total_loss = 0
        outputs_list = []
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = ssae(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            outputs_list.extend(outputs.argmax(dim=1).tolist())  # Collect max indices of softmax outputs
            
        ssae.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for batch_x, batch_y in train_loader:
                batch_x, batch_y = batch_x.to(device), batch_y.to(device)
                outputs = ssae(batch_x)
                _, predicted = outputs.max(1)
                total += batch_y.size(0)
                correct += predicted.eq(batch_y).sum().item()
        accuracy = correct / total

        # Print label distribution in predictions after each epoch
        print(f"Label distribution: {Counter(outputs_list)}")
        print(f'Fine-tuning Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {accuracy:.4f}')


        # # Check if model is better
        # if best_train_loss - total_loss > epsilon:
        #     best_train_loss = total_loss
        #     best_model = autoencoder.state_dict()
        #     epochs_no_improve = 0
        # else:
        #     epochs_no_improve += 1
        
        # # Check early stopping condition
        # if epochs_no_improve == patience:
        #     print("Early stopping!")
        #     break

    # Load best model
    # autoencoder.load_state_dict(best_model)
    return ssae

In [90]:
sub1 = df[df.subject == 'sub01']

In [93]:
np.stack(sub1.window_emg.to_numpy()).reshape(-1, 400).shape

(83700, 400)

In [94]:
def prepare_x(df):
    return np.stack(df.window_emg.to_numpy()).reshape(-1, 400)

In [95]:
def tensorize_x(X):
    return torch.tensor(X).to(torch.float32).to('mps')

In [100]:
tensorize_x(prepare_x(sub1)).shape

torch.Size([83700, 400])

In [105]:
X_train = tensorize_x(prepare_x(sub1))
y_train = sub1.label.tolist()

In [221]:
ssae = SSAE(400, 100, 50, 9)

In [113]:
a = DataLoader(TensorDataset(X_train.to(torch.float32), torch.FloatTensor(y_train)), batch_size=256, shuffle=True)

In [222]:
train_autoencoder(ssae.ae1, a, 500)

Epoch [1/500], Loss: 69.646914
Epoch [2/500], Loss: 43.281436
Epoch [3/500], Loss: 28.983236
Epoch [4/500], Loss: 21.005625
Epoch [5/500], Loss: 16.214978
Epoch [6/500], Loss: 13.123571
Epoch [7/500], Loss: 11.006210
Epoch [8/500], Loss: 9.485343
Epoch [9/500], Loss: 8.350480
Epoch [10/500], Loss: 7.477059
Epoch [11/500], Loss: 6.787497
Epoch [12/500], Loss: 6.231454
Epoch [13/500], Loss: 5.775030
Epoch [14/500], Loss: 5.394575
Epoch [15/500], Loss: 5.073184
Epoch [16/500], Loss: 4.798650
Epoch [17/500], Loss: 4.561741
Epoch [18/500], Loss: 4.355362
Epoch [19/500], Loss: 4.174293
Epoch [20/500], Loss: 4.014251
Epoch [21/500], Loss: 3.871837
Epoch [22/500], Loss: 3.744416
Epoch [23/500], Loss: 3.629770
Epoch [24/500], Loss: 3.526180
Epoch [25/500], Loss: 3.432157
Epoch [26/500], Loss: 3.346448
Epoch [27/500], Loss: 3.268021
Epoch [28/500], Loss: 3.196014
Epoch [29/500], Loss: 3.129745
Epoch [30/500], Loss: 3.068438
Epoch [31/500], Loss: 3.011693
Epoch [32/500], Loss: 2.958964
Epoch [33/

In [223]:
b = DataLoader(
    TensorDataset(
        torch.cat([ssae.ae1.encoder(batch[0].to('mps')).detach() for batch in a])
    ),
    batch_size=256,
    shuffle=True
)

In [224]:
train_autoencoder(ssae.ae2, b, 50)

Epoch [1/50], Loss: 3.240194
Epoch [2/50], Loss: 1.041446
Epoch [3/50], Loss: 0.337914
Epoch [4/50], Loss: 0.121366
Epoch [5/50], Loss: 0.056959
Epoch [6/50], Loss: 0.038076
Epoch [7/50], Loss: 0.032545
Epoch [8/50], Loss: 0.030916
Epoch [9/50], Loss: 0.030432
Epoch [10/50], Loss: 0.030286
Epoch [11/50], Loss: 0.030242
Epoch [12/50], Loss: 0.030229
Epoch [13/50], Loss: 0.030224
Epoch [14/50], Loss: 0.030223
Epoch [15/50], Loss: 0.030223
Epoch [16/50], Loss: 0.030222
Early stopping!


In [296]:
for i in ssae.parameters():
    print(i.shape)

torch.Size([100, 400])
torch.Size([100])
torch.Size([400, 100])
torch.Size([400])
torch.Size([50, 100])
torch.Size([50])
torch.Size([100, 50])
torch.Size([100])
torch.Size([9, 50])
torch.Size([9])


In [305]:
np.unique(df.motion)

array(['CH', 'EX', 'FL', 'GR', 'IN', 'OH', 'PR', 'RT', 'SU'], dtype=object)

In [302]:
next(iter(a))

[tensor([[ 0.0156,  0.0391, -0.0234,  ..., -0.0391, -0.0312, -0.0469],
         [-0.0938, -0.0312, -0.0156,  ...,  0.0000,  0.0781,  0.1094],
         [ 0.0078,  0.0078, -0.0156,  ..., -0.2656, -0.0312, -0.0156],
         ...,
         [-0.0078, -0.0078,  0.0000,  ..., -0.0078, -0.0078,  0.0000],
         [-0.0547, -0.0078, -0.0078,  ...,  0.0312,  0.0000, -0.0234],
         [-0.0078, -0.0156,  0.0000,  ..., -0.0156, -0.0234, -0.0078]],
        device='mps:0'),
 tensor([7., 3., 2., 8., 6., 1., 5., 4., 1., 5., 4., 8., 7., 0., 3., 0., 0., 7.,
         2., 7., 8., 7., 0., 3., 4., 0., 4., 8., 3., 7., 8., 3., 2., 7., 1., 5.,
         7., 5., 5., 1., 5., 2., 4., 8., 3., 3., 2., 3., 3., 3., 1., 1., 8., 3.,
         1., 0., 2., 6., 7., 5., 8., 7., 5., 2., 7., 3., 6., 6., 2., 0., 1., 2.,
         1., 7., 5., 3., 1., 2., 2., 5., 0., 0., 8., 2., 7., 5., 5., 2., 5., 5.,
         5., 8., 8., 8., 5., 8., 7., 8., 8., 3., 7., 4., 0., 7., 0., 8., 1., 7.,
         3., 5., 1., 2., 7., 0., 0., 6., 4., 6.,

In [300]:
ssae = SSAE(400, 100, 50, 9)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
ssae.to(device)

criterion = nn.CrossEntropyLoss()
parameters_to_optimize = list(ssae.ae1.encoder.parameters()) + \
                         list(ssae.ae2.encoder.parameters()) + \
                         list(ssae.classifier.parameters())
optimizer = optim.SGD(parameters_to_optimize, lr=.01, momentum=0.9)

for epoch in range(50):
    ssae.train()
    
    total_loss = 0
    outputs_list = []
    
    for inputs, labels in a:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = ssae(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        outputs_list.extend(outputs.argmax(dim=1).tolist())

    print(f"Label distribution: {Counter(outputs_list)}")
    print(f'Fine-tuning Epoch [{epoch+1}/{50}], Loss: {total_loss/len(a):.4f}')

Label distribution: Counter({4: 59562, 3: 24138})
Fine-tuning Epoch [1/50], Loss: 2.1975
Label distribution: Counter({3: 43843, 4: 27458, 5: 12399})
Fine-tuning Epoch [2/50], Loss: 2.1973
Label distribution: Counter({4: 49685, 3: 20889, 5: 13126})
Fine-tuning Epoch [3/50], Loss: 2.1973
Label distribution: Counter({3: 45072, 4: 30492, 5: 5564, 0: 2572})
Fine-tuning Epoch [4/50], Loss: 2.1973
Label distribution: Counter({4: 79634, 1: 1901, 0: 1185, 3: 752, 7: 228})
Fine-tuning Epoch [5/50], Loss: 2.1972
Label distribution: Counter({3: 32313, 7: 20463, 1: 18843, 0: 8774, 4: 3306, 5: 1})
Fine-tuning Epoch [6/50], Loss: 2.1972
Label distribution: Counter({3: 28872, 0: 17520, 6: 12608, 7: 9990, 5: 8995, 2: 5121, 4: 594})
Fine-tuning Epoch [7/50], Loss: 2.1973
Label distribution: Counter({5: 35388, 7: 21976, 6: 13649, 0: 12645, 4: 26, 8: 10, 2: 6})
Fine-tuning Epoch [8/50], Loss: 2.1973
Label distribution: Counter({7: 35627, 5: 26898, 4: 6170, 2: 5674, 6: 3761, 0: 2902, 3: 1927, 8: 740, 1: 1}

KeyboardInterrupt: 

In [287]:
ssae = SSAE(400, 100, 50, 9)
ssae = train_ssae(ssae, a, num_epochs=25)
ssae.load_state_dict(best_ssae)

Training autoencoder 1
Epoch [1/25], Loss: 68.661787
Epoch [2/25], Loss: 42.810947
Epoch [3/25], Loss: 28.779858
Epoch [4/25], Loss: 20.918176
Epoch [5/25], Loss: 16.177715
Epoch [6/25], Loss: 13.109153
Epoch [7/25], Loss: 11.002871
Epoch [8/25], Loss: 9.487515
Epoch [9/25], Loss: 8.355364
Epoch [10/25], Loss: 7.483260
Epoch [11/25], Loss: 6.794303
Epoch [12/25], Loss: 6.238494
Epoch [13/25], Loss: 5.781980
Epoch [14/25], Loss: 5.401326
Epoch [15/25], Loss: 5.079731
Epoch [16/25], Loss: 4.804880
Epoch [17/25], Loss: 4.567663
Epoch [18/25], Loss: 4.361047
Epoch [19/25], Loss: 4.179673
Epoch [20/25], Loss: 4.019310
Epoch [21/25], Loss: 3.876697
Epoch [22/25], Loss: 3.749069
Epoch [23/25], Loss: 3.634180
Epoch [24/25], Loss: 3.530437
Epoch [25/25], Loss: 3.436159
Training autoencoder 2
Epoch [1/25], Loss: 2.145752
Epoch [2/25], Loss: 0.694320
Epoch [3/25], Loss: 0.241208
Epoch [4/25], Loss: 0.098201
Epoch [5/25], Loss: 0.053025
Epoch [6/25], Loss: 0.038756
Epoch [7/25], Loss: 0.034245
Epo

<All keys matched successfully>

In [280]:
y_pred = ssae(X_train)

In [281]:
c = y_pred.max(1)[1].cpu().numpy()

In [282]:
np.unique(c)

array([6])